In [1]:
# this solution is for single instance, multi-node processing!


# !pip install warcio swifter html2text cchardet

In [2]:
import pandas as pd
import os
import numpy as np
import boto3
from warcio.archiveiterator import ArchiveIterator
import swifter
import timeit
import time
from io import BytesIO
from bs4 import BeautifulSoup, SoupStrainer
import lxml
import cchardet
import numpy as np
import botocore

In [3]:
covid_synonyms = ["covid", "SARS‑CoV‑2", "corona pandemic", "corona",  "covid 19" , "covid-19"
                  , "corona virus", "coronapandemie", "coronakrise", "SARS CoV 2",
                  "Wuhan virus", "pandemie", "pandemic", "2019 nCoV", "pandémie",
                  "pandemia", "Koronapandemie", "Korona", "Coronavirus",
                  "Coronapandemie", "Wuhan-Virus", "pandémie corona", "Virus de Wuhan", "NCoV 2019",
                  "pandemia de corona", "coronavirus", "coronapandemia", "cobiçado 19"
                  , "coronavírus", "Vírus Wuhan", "电晕大流行", "电晕", "冠状病毒", "冠状流感",
                  "日冕", "武汉病毒", "大流行", "大流行", "2019年nCoV",
                  "大流行", "कोरोना महामारी" , "कोरोना", "कोविद १ ९", "कोरोना", "ओ और ndemie",
                  "ओ nakrise", "वुहान वायरस", "ndemie", "महामारी", "コロナパンデミック", "コロナ",
                  "コロナウイルス", "コロナパンデミ", "コロナクリス", "武漢ウイルス", "パンデミー",
                  "パンデミック", "パンデミー", "パンデミック", "الاكليل", "جائحة الاكليل",
                  "مطمع ۱۹", "الفيروس التاجي", "التاج", "السارس", "فيروس ووهان",
                  "بانديمي", "جائحة", "ncov 2019", "2019 ncov", "пандемия короны",
                  "корона", "Ковид 19", "коронавирус", "ш и ndemie", "о в nakrise",
                  "Уханьский вирус", "ndemie", "пандемия", "2019 нКоВ", "ndemia"]

In [4]:
# df_reader = pd.read_csv('s3://cc-extract/top10subpages/2022/05/31/8f1d841c-8c64-4c84-85f8-e73c352e156f.csv', 
#                         chunksize=(os.cpu_count()-2)*10)

In [5]:
# def fetch_process_warc_records(input):
#     """Fetch all WARC records defined by filenames and offsets in input,
#     parse the records and the contained HTML, split the text into words
#     and emit pairs <word, 1>"""
#     times = {}
    
#     start = time.process_time()
#     boto_config = botocore.config.Config(max_pool_connections=os.cpu_count())
# #     boto_session = boto3.session.Session()
# #     s3client = boto_session.client('s3', 
# #                                    region_name='us-east-1', 
# #                                    use_ssl=False, 
# #                                    config=boto_config
# #                                   )
#     s3client = boto3.client('s3', 
#                            region_name='us-east-1', 
#                            use_ssl=False, 
#                            config=boto_config
#                            )
#     times['boto_startup'] = time.process_time() - start
     
#     url, warc_path, offset, end = input.split('<;>')
#     rangereq = 'bytes={}-{}'.format(offset, end)
    
#     start = time.process_time()
#     response = s3client.get_object(Bucket='commoncrawl',
#                                     Key=warc_path,
#                                     Range=rangereq)
#     times['s3_download'] = time.process_time() - start  
    
#     record_stream = BytesIO(response["Body"].read())

#     times['beautiful_soup_parsing'] = []
#     times['covid_paragraphs'] = []
#     covid_paragraphs = []
#     only_paragraphs = SoupStrainer('p')
#     for record in ArchiveIterator(record_stream):
#         page = record.content_stream().read()

#         start = time.process_time()
#         soup = BeautifulSoup(page, 'lxml', parse_only=only_paragraphs)
#         text = soup.get_text()
#         times['beautiful_soup_parsing'].append(time.process_time() - start)

#         start = time.process_time()         
#         paragraphs = text.split('\n')
#         covid_paragraphs += [paragraph for paragraph in paragraphs if any(ext.casefold() in paragraph.casefold() for ext in covid_synonyms)]
#         times['covid_paragraphs'].append(time.process_time() - start)

#     times['beautiful_soup_parsing'] = np.mean(times['beautiful_soup_parsing'])
#     times['covid_paragraphs'] = np.mean(times['covid_paragraphs'])
    
# #     print(f'Processing took {time.process_time() - start} seconds.')

#     res = '\n'.join(covid_paragraphs)
        
#     return {'res': res, 'times': times}

In [6]:
# start = time.process_time()
# results = []
# for df in df_reader:
#     df['input'] = df.url + '<;>' + df.warc_filename + '<;>' + df.warc_record_offset.apply(str) + '<;>' + df.warc_record_end.apply(str)
#     times = df.input.swifter.allow_dask_on_strings(enable=True).apply(lambda x: fetch_process_warc_records(x)['times'])
#     break
# print(time.process_time() - start)

In [7]:
# times = times.apply(pd.Series)
# times.mean()

In [8]:
def fetch_process_warc_records(batch):
    """Fetch all WARC records defined by filenames and offsets in batch,
    parse the records and the contained HTML, split the text into words
    and emit pairs <word, 1>"""
    batch['input'] = batch.url + '<;>' + batch.warc_filename + '<;>' + batch.warc_record_offset.apply(str) + '<;>' + batch.warc_record_end.apply(str)
    batch = batch.input.values
    
    s3client = boto3.client('s3', 
                           region_name='us-east-1', 
                           use_ssl=False
                           )
        
    only_paragraphs = SoupStrainer('p')
    
    results = []
    for x in batch:
        url, warc_path, offset, end = x.split('<;>')
        rangereq = 'bytes={}-{}'.format(offset, end)

        response = s3client.get_object(Bucket='commoncrawl',
                                        Key=warc_path,
                                        Range=rangereq)

        record_stream = BytesIO(response["Body"].read())
        
        covid_paragraphs = []
        for record in ArchiveIterator(record_stream):
            page = record.content_stream().read()

            soup = BeautifulSoup(page, 'lxml', parse_only=only_paragraphs)
            text = soup.get_text()

            paragraphs = text.split('\n')
            covid_paragraphs += [paragraph for paragraph in paragraphs if any(ext.casefold() in paragraph.casefold() for ext in covid_synonyms)]
            
        res = '\n'.join(covid_paragraphs)
        results.append(res)
        
    return results

In [9]:
df = pd.read_csv('s3://cc-extract/top10subpages/2022/05/31/8f1d841c-8c64-4c84-85f8-e73c352e156f.csv', nrows=1000)

In [10]:
import dask.dataframe as dd

start = time.process_time()
# Load the data with Dask instead of Pandas.
# ddf = dd.read_csv('s3://cc-extract/top10subpages/2022/05/31/8f1d841c-8c64-4c84-85f8-e73c352e156f.csv', 
#                   blocksize='64MB')
ddf = dd.from_pandas(df, npartitions=os.cpu_count()-2)

result = ddf.map_partitions(fetch_process_warc_records, meta=list)
result = result.compute(num_workers=os.cpu_count()-2)
print(time.process_time() - start)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


KeyError: 'credential_provider'